In [1]:
!nvidia-smi

Mon Dec  9 17:17:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**1. Install necessary packages**

In [2]:
!pip install -q transformers einops accelerate langchain langchain-community bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00


**2. Initialize the falcon model**

In [3]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline("text-generation",#task
                    model = model,
                    tokenizer = tokenizer,
                    torch_dtype = torch.bfloat16,
                    trust_remote_code = True,
                    device_map = "auto",
                    max_length = 200,
                    do_sample = True,
                    top_k = 10,
                    num_return_sequences= 1,
                    eos_token_id = tokenizer.eos_token_id
                    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [4]:
llm = HuggingFacePipeline(pipeline = pipeline,model_kwargs = {"temperature":0})

<ipython-input-4-a2dd8e2d53f6>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = pipeline,model_kwargs = {"temperature":0})


In [5]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7b1a5c1a5150>, model_kwargs={'temperature': 0})

**3. Create Prompt template**

In [6]:
from langchain import PromptTemplate, LLMChain

In [11]:
template = """
You are a helpfull AI assistant and provide the answer for the question asked politely.

{question}
Answer:
"""

prompt = PromptTemplate(template = template, input_variables = ["question"])

In [12]:
llm_chain = LLMChain(prompt=prompt,llm=llm)

In [13]:
question = "Who is the Prime Minister of India?"

print(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



You are a helpfull AI assistant and provide the answer for the question asked politely.

Who is the Prime Minister of India?
Answer: 

The current Prime Minister of India is Narendra Modi.


In [16]:
question = "Road map for Leanrning Gen AI?"
answer =llm_chain.invoke(question)
print(answer)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'question': 'Road map for Leanrning Gen AI?', 'text': '\nYou are a helpfull AI assistant and provide the answer for the question asked politely.\n\nRoad map for Leanrning Gen AI?\nAnswer: \n\n1) Understand the basics of Machine Learning, including supervised and unsupervised algorithms, supervised learning, and clustering algorithms.\n\n2) Learn about supervised and unsupervised learning algorithms.\n\n3) Get to know the various types of supervised and unsupervised learning algorithms.\n\n4) Learn about deep learning and neural networks.\n\n5) Understand the concepts and techniques associated with natural language processing, such as text and speech processing.\n\n6) Understand the importance of algorithms and their applications in real world problems.\n\n7) Learn about the principles and techniques of algorithm optimization, such as gradient-based optimization and genetic algorithms.\n\n8) Understand the concepts and techniques associated with deep learning, such as convolutional neu

In [17]:
question = "give me a famous nursery ryms"
answer =llm_chain.invoke(question)
print(answer)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'question': 'give me a famous nursery ryms', 'text': '\nYou are a helpfull AI assistant and provide the answer for the question asked politely.\n\ngive me a famous nursery ryms\nAnswer: \n\nHere are some famous nursery rhymes:\n\n1. Humpty Dumpty \n2. Twinkle, Twinkle, Little Star \n3. Mary, Mary, Quite Contrary \n4. Bohemian Rhapsody (by Queen) \n5. Hey, Diddle, Diddle (a song from a children’s book called "The Cat in the Hat")\n6. Little Boy Blue \n7. In the Jungle (by Elton John) \n8. Supercalifragilisticexpialidocious (by The Beatles)\n9. Humpty Dumpty \n10. The wheels on the bus (song).'}
